In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [30]:
from datasets import load_dataset

wiki_dataset = load_dataset("wikitext","wikitext-2-raw-v1")
wiki_dataset

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [31]:
wiki_train_dataset = wiki_dataset["train"]
wiki_train_dataset[0]

{'text': ''}

In [32]:
wiki_train_dataset.features

{'text': Value(dtype='string', id=None)}

**the 11th example (index 10) of the train split**.

In [34]:
wiki_train_dataset[10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [35]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-cased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [39]:
sequence2 = """ The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a character , and " Battle Potentials " , which are grown throughout the game and always grant boons to a character . To learn Battle Potentials , each character has a unique " Masters Table " , a grid @-@ based skill table that can be used to acquire and link different skills . Characters also have Special Abilities that grant them temporary boosts on the battlefield : Kurt can activate " Direct Command " and move around the battlefield without depleting his Action Point gauge , the character Reila can shift into her " Valkyria Form " and become invincible , while Imca can target multiple enemy units with her heavy weapon . \n"""

In [38]:
Text ="The game \'s battle system , the BliTZ system"
tokens = tokenizer.tokenize(Text)
print(tokens)

['The', 'game', "'", 's', 'battle', 'system', ',', 'the', 'B', '##li', '##T', '##Z', 'system']


In [25]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[1109, 1342, 112, 188, 2321, 1449, 117, 1103, 139, 2646, 1942, 5301, 1449]


**tokens of 11th token**

In [40]:
tokens = tokenizer.tokenize(sequence2)
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

['The', 'game', "'", 's', 'battle', 'system', ',', 'the', 'B', '##li', '##T', '##Z', 'system', ',', 'is', 'carried', 'over', 'directly', 'from', 'Val', '##ky', '##ira', 'Chronicles', '.', 'During', 'missions', ',', 'players', 'select', 'each', 'unit', 'using', 'a', 'top', '@', '-', '@', 'down', 'perspective', 'of', 'the', 'battlefield', 'map', ':', 'once', 'a', 'character', 'is', 'selected', ',', 'the', 'player', 'moves', 'the', 'character', 'around', 'the', 'battlefield', 'in', 'third', '@', '-', '@', 'person', '.', 'A', 'character', 'can', 'only', 'act', 'once', 'per', '@', '-', '@', 'turn', ',', 'but', 'characters', 'can', 'be', 'granted', 'multiple', 'turns', 'at', 'the', 'expense', 'of', 'other', 'characters', "'", 'turns', '.', 'Each', 'character', 'has', 'a', 'field', 'and', 'distance', 'of', 'movement', 'limited', 'by', 'their', 'Action', 'G', '##au', '##ge', '.', 'Up', 'to', 'nine', 'characters', 'can', 'be', 'assigned', 'to', 'a', 'single', 'mission', '.', 'During', 'gameplay

***Answer the question***

In [41]:
text = "This is a great [MASK]."
sequence_text_mask2 = """ The game \'s battle [MASK] , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a character , and " Battle Potentials " , which are grown throughout the game and always grant boons to a character . To learn Battle Potentials , each character has a unique " Masters Table " , a grid @-@ based skill table that can be used to acquire and link different skills . Characters also have Special Abilities that grant them temporary boosts on the battlefield : Kurt can activate " Direct Command " and move around the battlefield without depleting his Action Point gauge , the character Reila can shift into her " Valkyria Form " and become invincible , while Imca can target multiple enemy units with her heavy weapon . \n"""

In [45]:
import torch

inputs = tokenizer(sequence_text_mask2, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {sequence_text_mask2.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>>  The game 's battle mechanic , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters ' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a ch

In [43]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='distilbert-base-cased')
unmasker("Hello I'm a [MASK] model.")

[{'score': 0.15749548375606537,
  'token': 4633,
  'token_str': 'fashion',
  'sequence': "Hello I'm a fashion model."},
 {'score': 0.06042369827628136,
  'token': 1848,
  'token_str': 'professional',
  'sequence': "Hello I'm a professional model."},
 {'score': 0.025553911924362183,
  'token': 1648,
  'token_str': 'role',
  'sequence': "Hello I'm a role model."},
 {'score': 0.019409120082855225,
  'token': 10948,
  'token_str': 'celebrity',
  'sequence': "Hello I'm a celebrity model."},
 {'score': 0.017325857654213905,
  'token': 2235,
  'token_str': 'model',
  'sequence': "Hello I'm a model model."}]

In [44]:
unmasker(sequence_text_mask2)

[{'score': 0.19702057540416718,
  'token': 19459,
  'token_str': 'mechanic',
  'sequence': 'The game\'s battle mechanic, the BliTZ system, is carried over directly from Valkyira Chronicles. During missions, players select each unit using a top @ - @ down perspective of the battlefield map : once a character is selected, the player moves the character around the battlefield in third @ - @ person. A character can only act once per @ - @ turn, but characters can be granted multiple turns at the expense of other characters\'turns. Each character has a field and distance of movement limited by their Action Gauge. Up to nine characters can be assigned to a single mission. During gameplay, characters will call out if something happens to them, such as their health points ( HP ) getting low or being knocked out by enemy attacks. Each character has specific " Potentials ", skills unique to each character. They are divided into " Personal Potential ", which are innate skills that remain unaltere

***Encoding and Decoding***

In [15]:
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [16]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [17]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple
